In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# قراءة البيانات
orders = pd.read_csv(r"C:\Users\DELL\Desktop\after dit\orders_clean.csv")
order_items = pd.read_csv(r"C:\Users\DELL\Desktop\after dit\order_items_clean.csv")

In [ ]:

# تعويض القيم الناقصة في price_each من جدول المنتجات
products = pd.read_csv(r"C:\Users\DELL\Desktop\after dit\products_clean.csv")
order_items = order_items.merge(products[['product_id','price']], on='product_id', how='left')
order_items['price_each'] = order_items['price_each'].fillna(order_items['price'])
order_items.drop(columns=['price'], inplace=True)

# حساب total_amount للطلبات اللي ناقصة
calculated_totals = order_items.groupby('order_id').apply(lambda x: (x['quantity'] * x['price_each']).sum()).reset_index(name='calculated_total')
orders = orders.merge(calculated_totals, on='order_id', how='left')
orders['total_amount'] = orders['total_amount'].fillna(orders['calculated_total'])
orders.drop(columns=['calculated_total'], inplace=True)

# تنظيف التواريخ
def clean_date(date_str):
    if pd.isna(date_str) or date_str == 'INVALID_DATE':
        return pd.NaT
    date_formats = ['%Y-%m-%d','%b-%d-%Y','%m-%d-%Y','%m/%d/%Y','%d/%m/%Y']
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue
    return pd.NaT

orders['clean_order_date'] = orders['order_date'].apply(clean_date)


In [ ]:
# حفظ الملف المنظف
orders.to_csv(r"C:\Users\DELL\Desktop\after dit\orders_processed.csv", index=False)
